In [4]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#### Big, beautiful dataframe for TSAF

All series imported from APIs and starting in Jan. 2010

Weekly series will be averaged for the final dataframe

##### 1. Gasoline Stock

In [5]:
API_KEY = "JB0mjBHe8ebPVRafXPnetbaoNGigyEQTrYU3aoe3"
STOCKS_URL = "https://api.eia.gov/v2/petroleum/stoc/wstk/data/"

# 1. Setup Parameters for Stocks (using your provided structure)
params_stocks = {
    "api_key": API_KEY,
    "frequency": "weekly",
    "data[]": "value",
    "facets[duoarea][]": "NUS",
    "facets[process][]": "SAE",
    "facets[product][]": "EPM0",
    "facets[series][]": "WGTSTUS1",
    "start": "2010-01-01",
    "end": "2026-01-31",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000
}
response = requests.get(STOCKS_URL, params=params_stocks)
data = response.json()

#Extract the actual data rows into a Pandas DataFrame
gasoline_stock_weekly = pd.DataFrame(data['response']['data'])
gasoline_stock_weekly = gasoline_stock_weekly[['period', 'value']]

gasoline_stock_weekly['period'] = pd.to_datetime(gasoline_stock_weekly['period']) #makes sure the date is in Date format
gasoline_stock_weekly = gasoline_stock_weekly.set_index('period')
gasoline_data = gasoline_stock_weekly.sort_index(ascending=True)
gasoline_stock_weekly['value'] = pd.to_numeric(gasoline_stock_weekly['value'], errors='coerce')
gasoline_stock_weekly.columns = ["Gasoline_Stock"] #rename the columns
gasoline_stock_weekly.head()

,Gasoline_Stock
period,
2026-01-30,257898
2026-01-23,257213
2026-01-16,256990
2026-01-09,251013
2026-01-02,242036


#### 2. Gasoline prices (monthly)

In [6]:
API_KEY = "JB0mjBHe8ebPVRafXPnetbaoNGigyEQTrYU3aoe3"
URL = "https://api.eia.gov/v2/petroleum/pri/gnd/data/"

# Define parameters for Monthly Retail Gasoline Prices
params_monthly = {
    "api_key": API_KEY,
    "frequency": "monthly", # Switched from 'weekly' to 'monthly'
    "data[]": "value",
    "facets[series][]": "EMM_EPMR_PTE_NUS_DPG", 
    "start": "2010-01-01",
    "end": "2026-01-31",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000
}

response = requests.get(URL, params=params_monthly)
data = response.json()

# 3. DATAFRAME PROCESSING
gas_prices_monthly = pd.DataFrame(data['response']['data'])

# EIA Monthly data returns period as 'YYYY-MM'
gas_prices_monthly['Date'] = pd.to_datetime(gas_prices_monthly['period'])
gas_prices_monthly = gas_prices_monthly[['Date', 'value']].rename(columns={'value': 'Monthly_Gas_Price'})

# Sort and clean
gas_prices_monthly = gas_prices_monthly.sort_values('Date').set_index('Date')
gas_prices_monthly['Monthly_Gas_Price'] = pd.to_numeric(gas_prices_monthly['Monthly_Gas_Price'], errors='coerce')

print(gas_prices_monthly.head())

            Monthly_Gas_Price
Date                         
2010-01-01              2.715
2010-02-01              2.644
2010-03-01              2.772
2010-04-01              2.848
2010-05-01              2.836


##### 3. Gasoline prices (weekly)

In [7]:
API_KEY = "JB0mjBHe8ebPVRafXPnetbaoNGigyEQTrYU3aoe3"
# URL es el mismo que para montly data

params_weekly = {
    "api_key": API_KEY,
    "frequency": "weekly", # Set to weekly
    "data[]": "value",
    "facets[series][]": "EMM_EPMR_PTE_NUS_DPG", 
    "start": "2010-01-01",
    "end": "2026-01-31",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000
}

response = requests.get(URL, params=params_weekly)
data = response.json()

gas_prices_weekly = pd.DataFrame(data['response']['data'])

gas_prices_weekly['Date'] = pd.to_datetime(gas_prices_weekly['period'])
gas_prices_weekly = gas_prices_weekly[['Date', 'value']].rename(columns={'value': 'Weekly_Gas_Price'})

# Sort and clean
gas_prices_weekly = gas_prices_weekly.sort_values('Date').set_index('Date')
gas_prices_weekly['Weekly_Gas_Price'] = pd.to_numeric(gas_prices_weekly['Weekly_Gas_Price'], errors='coerce')

print(gas_prices_weekly.tail()) # Show the most recent dates

            Weekly_Gas_Price
Date                        
2025-12-29             2.811
2026-01-05             2.796
2026-01-12             2.779
2026-01-19             2.806
2026-01-26             2.853


#### 4. Inflation ex. food and regulated items (monthly).
 From [here](https://fred.stlouisfed.org/series/CORESTICKM159SFRBATL)

This is the percent change from a year before, not the index

In [8]:
FRED_API_KEY = "3213b1a82c8c665924ce8cc536c30814"
SERIES_ID = "CORESTICKM159SFRBATL"
URL = "https://api.stlouisfed.org/fred/series/observations"

# Parameters for FRED API
params = {
    "api_key": FRED_API_KEY,
    "series_id": SERIES_ID,
    "file_type": "json",
    "observation_start": "2010-01-01",
    "observation_end": "2026-01-31"
}

response = requests.get(URL, params=params)
data = response.json()

inflation = pd.DataFrame(data['observations'])

inflation['Date'] = pd.to_datetime(inflation['date'])
inflation['Value'] = pd.to_numeric(inflation['value'], errors='coerce')

# Select relevant columns and set index
inflation = inflation[['Date', 'Value']].rename(columns={'Value': 'Core_Sticky_CPI'})
inflation = inflation.set_index('Date').sort_index()

print(inflation.head())

            Core_Sticky_CPI
Date                       
2010-01-01         1.170764
2010-02-01         1.048684
2010-03-01         0.925503
2010-04-01         0.869413
2010-05-01         0.793302


##### 5. Crude Oil imports into the US (monthly)

Millions of barrels
The original data shows several "types" of crude oil. I will download all of them and later we can add them or see if each one separately is better for the model.

In [9]:
API_KEY = "JB0mjBHe8ebPVRafXPnetbaoNGigyEQTrYU3aoe3" # Note: Be careful sharing API keys publicly!
URL = "https://api.eia.gov/v2/crude-oil-imports/data/"

params = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data[0]": "quantity",
    "facets[destinationType][]": "US",
    "start": "2010-01",
    "end": "2025-11",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "length": 5000
}


all_data = [] # Empty list to store all our batches of data
offset = 0

#loop through different types of crude oil:
while True:
    params["offset"] = offset # Update the offset for each loop
    response = requests.get(URL, params=params)
    json_data = response.json()
    
    # Check if the response contains data
    if 'response' in json_data and 'data' in json_data['response']:
        batch_data = json_data['response']['data']
        
        # If the batch is empty, we've reached the end of the data
        if not batch_data:
            break
            
        all_data.extend(batch_data)     
        # If the API returned fewer records than our max length, we are on the last page
        if len(batch_data) < 5000:
            break
            
        # Increase the offset to get the next page on the next loop
        offset += 5000 
    else:
        print(f"Error or end of data at offset {offset}. Check API response.")
        break

df_raw = pd.DataFrame(all_data)
df_raw['Date'] = pd.to_datetime(df_raw['period'])
df_raw['quantity'] = pd.to_numeric(df_raw['quantity'], errors='coerce')

# Group by Date and Grade (this sums up all countries for each grade)
df_grouped = df_raw.groupby(['Date', 'gradeName'])['quantity'].sum().reset_index()

# Pivot so each grade becomes a column
oil_imports_df = df_grouped.pivot(index='Date', columns='gradeName', values='quantity')

# DYNAMIC RENAMING:
oil_imports_df.columns = [f"import_{col.lower().replace(' ', '_')}" for col in oil_imports_df.columns]

# Final cleanup
oil_imports_df = oil_imports_df.fillna(0).sort_index()

print(oil_imports_df.head())


            import_heavy_sour  import_heavy_sweet  import_light_sour  \
Date                                                                   
2010-01-01             430340               30820              66372   
2010-02-01             401212               28848              55060   
2010-03-01             475960               20468              69724   
2010-04-01             461592               24848              70352   
2010-05-01             518760               28408              62404   

            import_light_sweet  import_medium  
Date                                           
2010-01-01              219020         351720  
2010-02-01              203584         323896  
2010-03-01              209576         420404  
2010-04-01              208680         445872  
2010-05-01              199528         432484  


##### 6. Miles travelled by vehicles

Miles travelled by vehicles serves as a proxy of gasoline demand.

Originally the data is from US Federal Highway Administration, but is available at. [FRED](https://fred.stlouisfed.org/series/TRFVOLUSM227NFWA)

It has both a seasonally adjusted and not-seasonally adjusted series. We will download both.


#### 6.1: Not seasonally adjusted

In [10]:
FRED_API_KEY = "3213b1a82c8c665924ce8cc536c30814"
SERIES_ID = "TRFVOLUSM227NFWA"
URL = "https://api.stlouisfed.org/fred/series/observations"

params = {
    "api_key": FRED_API_KEY,  #same as for inflation
    "series_id": SERIES_ID,
    "file_type": "json",
    "observation_start": "2010-01-01",
    "observation_end": "2026-01-31"
}

response = requests.get(URL, params=params)
data = response.json()

Miles_NSA = pd.DataFrame(data['observations'])

# Clean dates and values (FRED uses 'date' and 'value' as keys)
Miles_NSA['Date'] = pd.to_datetime(Miles_NSA['date'])
Miles_NSA['value'] = pd.to_numeric(Miles_NSA['value'], errors='coerce')

# Format and Rename
Miles_NSA = Miles_NSA[['Date', 'value']].rename(columns={'value': 'miles_traveled'})
Miles_NSA = Miles_NSA.set_index('Date').sort_index()

print(f"Retrieved {len(Miles_NSA)} months of travel data.")
print(Miles_NSA.head())

Retrieved 192 months of travel data.
            miles_traveled
Date                      
2010-01-01        220839.0
2010-02-01        210635.0
2010-03-01        254238.0
2010-04-01        253936.0
2010-05-01        256927.0


#### 6.2: Seasonally adjusted series

In [11]:
FRED_API_KEY = "3213b1a82c8c665924ce8cc536c30814"
SERIES_ID = "TRFVOLUSM227SFWA"
URL = "https://api.stlouisfed.org/fred/series/observations"

params = {
    "api_key": FRED_API_KEY,  #same as for inflation
    "series_id": SERIES_ID,
    "file_type": "json",
    "observation_start": "2010-01-01",
    "observation_end": "2026-01-31"
}

response = requests.get(URL, params=params)
data = response.json()

# FRED data is nested in the 'observations' key
if 'observations' in data:
    Miles_SA = pd.DataFrame(data['observations'])
    
    # Clean dates and values (FRED uses 'date' and 'value' as keys)
    Miles_SA['Date'] = pd.to_datetime(Miles_SA['date'])
    Miles_SA['value'] = pd.to_numeric(Miles_SA['value'], errors='coerce')
    
    # Format and Rename
    Miles_SA = Miles_SA[['Date', 'value']].rename(columns={'value': 'miles_traveled'})
    Miles_SA = Miles_SA.set_index('Date').sort_index()
    
    print(f"Retrieved {len(Miles_SA)} months of travel data.")
    print(Miles_SA.head())

Retrieved 192 months of travel data.
            miles_traveled
Date                      
2010-01-01        242519.0
2010-02-01        241803.0
2010-03-01        248076.0
2010-04-01        249112.0
2010-05-01        247042.0


#### 7. Oil production (total production per month, thousands of barrels)

In [12]:
API_KEY = "JB0mjBHe8ebPVRafXPnetbaoNGigyEQTrYU3aoe3"
URL = "https://api.eia.gov/v2/petroleum/crd/crpdn/data/"

params_prod = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data[0]": "value",
    "facets[product][]": "EPC0",
    "facets[duoarea][]": "NUS",
    "facets[series][]": "MCRFPUS1", 
    "start": "2010-01",
    "end": "2025-11",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000
}

response = requests.get(URL, params=params_prod)
json_res = response.json()

df_oil_prod = pd.DataFrame(json_res['response']['data'])

# Clean and standardize
df_oil_prod['Date'] = pd.to_datetime(df_oil_prod['period'])
df_oil_prod['oil_production'] = pd.to_numeric(df_oil_prod['value'], errors='coerce')

# Final Selection
oil_production = df_oil_prod[['Date', 'oil_production']].set_index('Date').sort_index()

print(oil_production.tail())

            oil_production
Date                      
2025-07-01          424926
2025-08-01          428114
2025-09-01          414845
2025-10-01          429781
2025-11-01          413457


### 8. Merging data

For weekly data, calculate the mean to match the monthly seasonality

In [13]:
miles_nsa_clean = Miles_NSA.rename(columns={Miles_NSA.columns[0]: 'miles_traveled_nsa'})
miles_sa_clean = Miles_SA.rename(columns={Miles_SA.columns[0]: 'miles_traveled_sa'})

# Resample weekly data to monthly
gas_stock_monthly = gasoline_stock_weekly.resample('MS').mean()
gas_price_weekly_avg = gas_prices_weekly.resample('MS').mean()

monthly_dfs = [
    gas_stock_monthly,   
    gas_prices_monthly,    
    gas_price_weekly_avg,  
    inflation, 
    oil_production,           
    oil_imports_df,
    miles_nsa_clean,      
    miles_sa_clean        
]

# Using 'outer' join to preserve all dates across the series
master_df = pd.concat(monthly_dfs, axis=1)

master_df.tail()

,Gasoline_Stock,Monthly_Gas_Price,Weekly_Gas_Price,Core_Sticky_CPI,oil_production,import_heavy_sour,import_heavy_sweet,import_light_sour,import_light_sweet,import_medium,miles_traveled_nsa,miles_traveled_sa
2025-09-01,218727.50,3.166,3.16560,3.322171,414845.0,472328.0,12460.0,43628.0,71756.0,167984.0,279611.0,278146.0
2025-10-01,214269.00,3.060,3.05975,3.077827,429781.0,434736.0,5040.0,24084.0,56276.0,211192.0,291537.0,277645.0
2025-11-01,209195.25,3.050,3.04950,2.954815,413457.0,418460.0,13248.0,13964.0,57076.0,190868.0,263087.0,277198.0
2025-12-01,227317.25,2.894,2.89440,3.006319,NaN,NaN,NaN,NaN,NaN,NaN,265776.0,277183.0
2026-01-01,253030.00,2.809,2.80850,2.979751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# CHECK FOR MISSING VALUES
na_counts = master_df.isna().sum()
print(na_counts)

Gasoline_Stock        0
Monthly_Gas_Price     0
Weekly_Gas_Price      0
Core_Sticky_CPI       0
oil_production        2
import_heavy_sour     2
import_heavy_sweet    2
import_light_sour     2
import_light_sweet    2
import_medium         2
miles_traveled_nsa    1
miles_traveled_sa     1
dtype: int64


Missing values are at the end of the dataframe because some series are not available until jan 2026.

We can fill the missing values using ffill()
Or we can just model without the last rows. **(model until November 2025)**

In [15]:
# Fill missing values using ffill (carrying forward the last available value)
master_df = master_df.ffill()

### 9. Adding up all imports

In [16]:
importaciones = [
    'import_heavy_sour', 
    'import_heavy_sweet', 
    'import_light_sour', 
    'import_light_sweet', 
    'import_medium'
]
master_df['total_imports'] = master_df[importaciones].sum(axis=1)
print(master_df[['total_imports'] + importaciones].tail())

            total_imports  import_heavy_sour  import_heavy_sweet  \
2025-09-01       768156.0           472328.0             12460.0   
2025-10-01       731328.0           434736.0              5040.0   
2025-11-01       693616.0           418460.0             13248.0   
2025-12-01       693616.0           418460.0             13248.0   
2026-01-01       693616.0           418460.0             13248.0   

            import_light_sour  import_light_sweet  import_medium  
2025-09-01            43628.0             71756.0       167984.0  
2025-10-01            24084.0             56276.0       211192.0  
2025-11-01            13964.0             57076.0       190868.0  
2025-12-01            13964.0             57076.0       190868.0  
2026-01-01            13964.0             57076.0       190868.0  


### 10. Creating "supply" column = imports + local production 

In [17]:
master_df['total_crude_supply'] = master_df['oil_production'] + master_df['total_imports']

### 11. Creating Covid dummy

In [18]:
covid_start = '2020-03-01'
covid_end = '2021-06-30'

master_df['covid_dummy'] = np.where(
    (master_df.index >= covid_start) & (master_df.index <= covid_end), 
    1, 
    0
)

master_df.tail()

,Gasoline_Stock,Monthly_Gas_Price,Weekly_Gas_Price,Core_Sticky_CPI,oil_production,import_heavy_sour,import_heavy_sweet,import_light_sour,import_light_sweet,import_medium,miles_traveled_nsa,miles_traveled_sa,total_imports,total_crude_supply,covid_dummy
2025-09-01,218727.50,3.166,3.16560,3.322171,414845.0,472328.0,12460.0,43628.0,71756.0,167984.0,279611.0,278146.0,768156.0,1183001.0,0
2025-10-01,214269.00,3.060,3.05975,3.077827,429781.0,434736.0,5040.0,24084.0,56276.0,211192.0,291537.0,277645.0,731328.0,1161109.0,0
2025-11-01,209195.25,3.050,3.04950,2.954815,413457.0,418460.0,13248.0,13964.0,57076.0,190868.0,263087.0,277198.0,693616.0,1107073.0,0
2025-12-01,227317.25,2.894,2.89440,3.006319,413457.0,418460.0,13248.0,13964.0,57076.0,190868.0,265776.0,277183.0,693616.0,1107073.0,0
2026-01-01,253030.00,2.809,2.80850,2.979751,413457.0,418460.0,13248.0,13964.0,57076.0,190868.0,265776.0,277183.0,693616.0,1107073.0,0


Export CSV - added by nyssa

In [25]:
master_df.to_csv('master_df.csv')

#### "Data challenges" to include in ppt: 

##### 1. Matching granularity.

Stocks are weekly but other variables are monthly.

Options:

- end of month
- 4 week average (we used this one)

##### 2. Missing values:

- using ffill() instead of interpolation. If using interpolation it would be like "seeing" the future.

##### 3. (to-do) Check if we should sum the different oil imports or leave them like separated.

##### 4. Dummy variable for Covid
